In [12]:
'''
#reset environment
def reset(self):

def step( self, action ):
  return obs, reward, flgdone, extra_info
'''

import carla
import math
import random
import time
import numpy as np
import inspect
import cv2
import math

#riferimento al tutorial
#https://carla.readthedocs.io/en/latest/tuto_first_steps/
#https://www.youtube.com/watch?v=pONr1R1dy88

#size di default del sensore
IM_WIDTH = 500
IM_HEIGHT = 500

SECONDS_PER_EPISODE = 10

#display camera da Carla
SHOW_PREVIEW = False

class CarEnv:
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0 #FULL TURN
    im_width = IM_WIDTH
    im_height = IM_HEIGHT
    front_camera = None
    
    def __init__(self):
        self.client = carla.Client("localhost", 2000);
        self.client.set_timeout(2.0)
        self.world = self.client.get_world()
        blueprint_library = self.world.get_blueprint_library()
        self.model = self.blueprint_library.find('vehicle.lincoln.mkz_2020')

    #reinizializza la classe con oggetti e sensori di default, svuotando le liste di attrori e azioni
    def reset(self):
        self.collision_hist = [] # il sensore di collisione ritorna un set di eventi di collisione, es: se il paraurti tocca il suolo a causa di un dosso
        self.actor_list = []
        
        self.transform = random.choise(self.world.get_map().get_spawn_points())
        self.vehicle = self.world.spawn_actor(self.model, self.transform)
        self.actor.list.append(self.vehicle)
        
        self.rgb_cam = self.blueprint_library.find('sensor.camera.rgb')
        self.rgb_cam.set_attribute('image_size_x', f"{IM_WIDTH}")
        self.rgb_cam.set_attribute('image_size_y', f"{IM_HEIGHT}")
        self.rgb_cam.set_attribute("fov","110")
        
        #posizione sensori
        transform = carla.Transform(carla.Location( x=2.5, z=0.7))
        
        #camera rgb
        self.sensor = self.world.spawn_actor(self.rgb_cam, transform, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process(img_data))
        
        #trick per rendere più reattiva l'auto
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(4)
        
        #definizione sensore di collisione
        colsensor = self.blueprint_library.find('sensor.other.collision')
        self.colsensor = self.world.spawn_actor(colsensor, transform, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))

        #controlla che la camera sia inizializzata, altrimeti attende
        while self.front_camera is None:
            time.sleep(0.01)

        self.espisode_start = time.time() #in teoria vogliamo episodi di 10 secondi
        #trick per rendere più reattiva l'auto
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        return self.front_camera
    
    #reinizializza la classe con oggetti e sensori di default, svuotando le liste di attrori e azioni
    def collision_data(self, event):
        self.collision_hist.append(event)

    #cattura i raw data del sensore e li trasforma in immagini priettate in imgshow di opencv
    def process_img(image):
        i = np.array(image.raw_data) #raw.data è un array flat
        #print(i.shape)
        i2 = i.reshape((self.im_height, self.im_width, 4))
        i3 = i2[:, :, :3] #intera altezza, larghezza, i primi tre valori sono rgb
        #print(i3)
        if self.SHOW_CAM:
            cv2.imshow("RGB_SENS",i3) #NON DARE UN TITOLO SE NO NON FUNZIONA
            cv2.waitKey(1) #ms: aggiornamento finestra a 0 non funziona
        self.front_camera =  i3
        return i3/255.0 #normalizzazione valori tra 0 e 1

    def step(self, action):
        if action == 0:
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=-1*self.STEER_AMT)) #decrementa la quantità di sterzo verso sinistra
        elif action == 1:
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0)) # dritto
        elif action == 2:
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=1*self.STEER_AMT)) #incrementa la quantità di sterzo verso destra
        
        #se non collide 
        v = self.vehicle.get_velocity()
        #conversione a kilometri/h
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))

        #calcolo reward, controllo se c'è almeno una collisione
        if len(self.collision_hist) != 0:
            done = True
            reward = -200 #per ora usiamo questo valore
        elif kmh < 50:
            done = False
            reward = -1
        else: #SE 50 KMH AUMENTO REAWRD DI 1
            done = False
            reward = 1

        if self.episode_start + SECONDS_PER_EPISODE < TIME.TIME():
            done = True
        #controllo che la dimensione degli steps sia nei 10 seocndi
        return self.front_camera, reward, done, None




In [1]:
for a in actors:
    a.destroy()
cv2.destroyAllWindows()
#for v in world.get_actors().filter('*vehicle*'):
#    v.destroy()

NameError: name 'actors' is not defined